# OpenAI Assistants API - RAG in ReAct

Praktični del 5. delavnice v sklopu Akademije umetne inteligence za poslovne aplikacije.

V tej beležki bomo uporabili OpenAI Assistants API za odgovarjanje na vprašanja o naših podatkih (RAG) ter za izvajanje naših funkcij (podobno kot ReAct cikel).

OpenAI asistenti so podobni GPT-jem, ki smo jih spoznali zadnjič, le da tečejo v vaši aplikacija in komunicirajo z OpenAI-jem le prek API-ja (za razliko od GPT-jev, ki delujejo v ChatGPT aplikaciji). OpenAI asistenti imajo na voljo več orodji (ki jih izvajajo v ciklu podobnem ReAct-u):
* File Search - lahko ga uporabljamo za RAG;
* Code Intepreter (izvajanje kode, ki jo napiše ChatGPT);
* lastna orodja (funkcije, ki jih spišemo sami in damo na voljo ChatGPT-ju).



In [ ]:
%%capture
!pip install --upgrade openai

In [ ]:
import openai

## RAG

V tem delu beležke bomo s pomočjo RAG-a in OpenAI asistentov zgradili chatbot-a, ki lahko odgovarja na vprašanja o pralnem stroju.

In [ ]:
client = openai.OpenAI(api_key="")

In [ ]:
assistant = client.beta.assistants.create(
  name="Washing Machine Expert",
  instructions="You are a Washing Machine Expert. You use the knowledge based with File Search to answer wachine machines related questions. Always consult the knowledge base before answering questions.Do not answer on your own. If the question is not about the knowledge base, tell the user that you can't help with that question.",
  model="gpt-3.5-turbo",
  tools=[{"type": "file_search"}],
)

Naredimo vektorsko bazo:

In [ ]:
# vector_store = client.beta.vector_stores.create(name="Washing Machine Instructions")

# file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
#   vector_store_id=vector_store.id, files=[open("path-to-washing-machine-instructions-pdf", "rb")]
# )

In [ ]:
for vector_store in client.beta.vector_stores.list():
    print(f"{vector_store.name}: {vector_store.id}, {vector_store.file_counts}")

In [ ]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": ["vs_0PDzwY4ZsvwJRs0ORmoXVTAE"]}},
)

Naredimo pomožno funkcijo, da bomo lažje klicali OpenAI Assistants API:

In [ ]:
thread = None

def ask_assistant(question: str) -> str:
    global thread
    if thread is None:
        thread = client.beta.threads.create()

    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=question
    )

    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id, assistant_id=assistant.id
    )

    messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

    message_content = messages[0].content[0].text
    # unstable API...
    if type(message_content) != dict:
        return message_content.value

    for i, annotation in enumerate(message_content["annotations"]):
        message_content["value"] = message_content["value"].replace(annotation["text"], f"[Ref. {i}]")

    return message_content["value"]

In [ ]:
print(ask_assistant("My washing machine is not working, what can I do?"))

## Tool/function calling

V tem delu beležke bomo s pomočjo OpenAI Assistant API-ja naredili chatbot-a, s katerim bomo lahko upravljali svoj TODO seznam.

In [ ]:
import json

In [ ]:
class TODOList:
    def __init__(self):
        self.tasks = set()

    def add(self, task: str):
        self.tasks.add(task)

    def remove(self, task: str):
        self.tasks.remove(task)

    def __str__(self):
        return "TODO:\n - " + "\n - ".join(self.tasks)

In [ ]:
todo_add_description = {
    "type": "function",
    "function": {
        "name": "todo_add",
        "description": "Add a task to the user TODO list",
        "parameters": {
            "type": "object",
            "properties": {
                "task": {
                    "type": "string",
                    "description": "The task that the user wants to add to their TODO list, e.g., 'Buy milk'"
                }
            },
            "required": ["task"]
        }
    }
}


todo_remove_description = {
    "type": "function",
    "function": {
        "name": "todo_remove",
        "description": "Remove a task from the user TODO list, if user wants to remove it or has already completed it",
        "parameters": {
            "type": "object",
            "properties": {
                "task": {
                    "type": "string",
                    "description": "The task that the user wants to remove from their TODO list, make sure that it's exactly the same as the task on the list. If you are not sure, check the list first."
                }
            },
            "required": ["task"]
        }
    }
}


todo_get_description = {
    "type": "function",
    "function": {
        "name": "todo_get",
        "description": "Get a string of user TODO list with all the tasks."
    }
}

In [ ]:
assistant = client.beta.assistants.create(
  name="TODO List Assistant",
  instructions="You are a TODO List Assistant. Use the provided functions to manage a user's TODO list.",
  model="gpt-3.5-turbo",
  tools=[todo_add_description, todo_remove_description, todo_get_description]
)

In [ ]:
todo_list = TODOList()
thread = None

def ask_assistant(question: str) -> str:
    global thread
    if thread is None:
        thread = client.beta.threads.create()

    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=question
    )

    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id, assistant_id=assistant.id
    )

    # using the tools
    while run.required_action is not None:
        tool_outputs = []

        for tool in run.required_action.submit_tool_outputs.tool_calls:
            print(f"Using tool: {tool.function.name}")
            output = ""
            if tool.function.name == "todo_add":
                try:
                    todo_list.add(**json.loads(tool.function.arguments))
                    output = "Successfully added"
                except Exception as e:
                    output = str(e)

            elif tool.function.name == "todo_remove":
                try:
                    todo_list.remove(**json.loads(tool.function.arguments))
                    output = "Successfully removed"
                except Exception as e:
                    output = str(e)

            elif tool.function.name == "todo_get":
                try:
                    output = str(todo_list)
                except Exception as e:
                    output = str(e)

            tool_outputs.append({"tool_call_id": tool.id, "output": output})

        # try submitting tool outputs
        if tool_outputs:
            try:
                run = client.beta.threads.runs.submit_tool_outputs_and_poll(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
                )
            except Exception as e:
                print("Failed to submit tool outputs:", e)
            else:
                print("No tool outputs to submit.")

    print("--------------------")
    if run.status == "completed":
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print(messages.data[0].content[0].text.value)
    else:
        print(run.status)

In [ ]:
ask_assistant("Do I have anything on my TODO?")

In [ ]:
ask_assistant("Add buying milk to my TODO?")

In [ ]:
print(todo_list)

In [ ]:
ask_assistant("What's on my TODO list?")

In [ ]:
ask_assistant("I have bought the milk.")